In [29]:
import pandas as pd
from sshtunnel import SSHTunnelForwarder
from pymongo import MongoClient
from pymongo.errors import OperationFailure, DuplicateKeyError
import names
from tqdm import tqdm

In [19]:
profile = pd.read_csv('../YuEBaoCashflows/Purchase Redemption Data/user_profile_table.csv')

In [20]:
def generate_customer_dict(user_id, profile_tbl):
    customer_dict = dict()
    customer_row = profile_tbl.loc[profile_tbl['user_id'] == user_id].iloc[0]
    customer_dict['user_id'] = int(customer_row['user_id'])
    customer_dict['gender'] = 'male' if int(customer_row['sex']) == 1 else 'female'
    customer_dict['name'] = names.get_full_name(gender=customer_dict['gender'])
    customer_dict['city'] = customer_row['city']
    customer_dict['constellation'] = customer_row['constellation']
    return customer_dict

In [5]:
server = SSHTunnelForwarder(
    "35.223.217.140",
    ssh_username="apple",
    ssh_pkey="/Users/apple/Desktop/CSSE433/project-cli-keys/key-instance-1",
    ssh_private_key_password="csse433",
    remote_bind_address=("10.128.0.6", 40000)
)
server.start()
host = "localhost"
port = server.local_bind_port
try:
    cli = MongoClient(f'mongodb://{host}:{port}/?authSource=admin')
    cli.server_info()
except OperationFailure as err:
    print(err)

In [22]:
db = cli['yuebao']
customers = db['customer']

In [26]:
try:
    customers.insert(generate_customer_dict(1, profile))
except DuplicateKeyError:
    print("duplicated id")

duplicated id


In [31]:
for uid in tqdm(profile['user_id'].values):
    customers.insert_one(generate_customer_dict(uid, profile))

100%|██████████| 28041/28041 [28:52<00:00, 16.18it/s]  
